In [ ]:
with open("test.html", "w", encoding = "utf8") as f:
    f.write(res.text)

## 기본 틀 만들기

In [ ]:
import requests 
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from datetime import datetime,timedelta


# requests 활용
url = "https://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&stn=0&type=t11&reg=100&tm=2020.01.01.14%3A00&x=16&y=5"
res = requests.get(url)
print("응답코드:", res.status_code) ## 200이면 정상, 403이면 접근권한 없음
    
soup = BeautifulSoup(res.text,"lxml")


# 지역별 기온 표 전체를 받기
table = soup.select('table') 
table = str(table[0])
table_df_list = pd.read_html(table)
table_df = table_df_list[0]

# 날짜 받기
date = soup.select_one('#observation_text')
date = date["value"]
time = datetime(int(date[:4]),int(date[5:7]),int(date[8:10]),int(date[11:13]),int(date[14:16]))
print(time)
# 지역 수 받기
NoOfCities = len(table_df)

# 1년 전체 횟수 데이터 틀 만들기
Result = table_df.iloc[:,:2]
Result.rename(columns = {'01H' : 'Rep'}, inplace = True)
Result["Rep"] = 0

# 오전 8시부터 오후 6시까지만 추출하기
Worktime = pd.concat([table_df.iloc[:,0],table_df.iloc[:,8:19]],axis = 1)

# 기온이 영하인 시간대만 추출하기
Cold = Worktime[Worktime.iloc[:,1:]<0]
Cold = np.array(Cold)

# 각 지역별로 기온이 영하인 총 시간 구하기
for i in range(NoOfCities):
    num = (Cold[i]<0).sum()
    Result.iloc[i,1] += num
Result

## For문을 통한 자동화

In [ ]:
# 다른 날짜로 이동해서 같은 거 반복하기
while(date != "2020.12.31.14:00"):
    Date = str(time + timedelta(days = 1))
    Date = Date.replace("-",".")
    Date = Date.replace(" ",".")
    Date = Date[:-6]

    url = "https://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&stn=0&type=t11&reg=100&tm="+Date+"%3A00&x=16&y=5"
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,"lxml")
    
    # 지역별 기온 표 전체를 받기
    table = soup.select('table.table_develop3') 
    table = str(table)
    table_df_list = pd.read_html(table)
    table_df = table_df_list[0]

    # 날짜 받기
    date = soup.select_one('#observation_text')
    date = date["value"]
    print(date)
    time = datetime(int(date[:4]),int(date[5:7]),int(date[8:10]),int(date[11:13]),int(date[14:16]))


    # 오전 8시부터 오후 6시까지만 추출하기
    Worktime = pd.concat([table_df.iloc[:,0],table_df.iloc[:,8:19]],axis = 1)

    # 기온이 영하인 시간대만 추출하기
    Cold = Worktime[Worktime.iloc[:,1:]<0]
    Cold = np.array(Cold)

    # 각 지역별로 기온이 영하인 총 시간 구하기
    for i in range(NoOfCities):
        num = (Cold[i]<0).sum()
        Result.iloc[i,1] += num


## 횟수 곱하고 엑셀 파일 만들기

In [ ]:
Result = np.array(Result) 
Result[:,1] = Result[:,1]* 720
Result = pd.DataFrame(Result)

Result.to_excel("Repetitions.xlsx")


## 월별 데이터

In [3]:
month = 1

writer = pd.ExcelWriter('월별현황.xlsx', engine='xlsxwriter')


# 다른 날짜로 이동해서 같은 거 반복하기
while(date != "2021.01.01.14:00"):
        
    Date = str(time + timedelta(days = 1))
    Date = Date.replace("-",".")
    Date = Date.replace(" ",".")
    Date = Date[:-6]

    url = "https://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&stn=0&type=t11&reg=100&tm="+Date+"%3A00&x=16&y=5"
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,"lxml")
    
    # 지역별 기온 표 전체를 받기
    table = soup.select('table.table_develop3') 
    table = str(table)
    table_df_list = pd.read_html(table)
    table_df = table_df_list[0]

    # 날짜 받기
    date = soup.select_one('#observation_text')
    date = date["value"]
    print(date)
    time = datetime(int(date[:4]),int(date[5:7]),int(date[8:10]),int(date[11:13]),int(date[14:16]))
    
    # 한 달이 넘어갈 때 -> 개월 수 높인 다음에 엑셀 시트에 저장.
    if int(date[5:7]) > month or int(date[:4]) == 2021:
        
        sheetname = str(month)+"월"
        
        Result = np.array(Result) 
        Result[:,1] *= 720
        Result = pd.DataFrame(Result)
        
        Result.to_excel(writer, sheet_name= sheetname)
        month += 1
        Result.iloc[:,1] = 0



    # 오전 8시부터 오후 6시까지만 추출하기
    Worktime = pd.concat([table_df.iloc[:,0],table_df.iloc[:,8:19]],axis = 1)

    # 기온이 영하인 시간대만 추출하기
    Cold = Worktime[Worktime.iloc[:,1:]<0]
    Cold = np.array(Cold)

    # 각 지역별로 기온이 영하인 총 시간 구하기
    for i in range(NoOfCities):
        num = (Cold[i]<0).sum()
        Result.iloc[i,1] += num
    
    print(Result)
writer.save()

2020.01.14.14:00
     지점  Rep
0    강릉    3
1   강진군    9
2    강화   50
3    거제    2
4    거창   29
..  ...  ...
90   합천   30
91   해남   15
92   홍성   21
93   홍천   57
94  흑산도    0

[95 rows x 2 columns]


C:\Users\erich\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in less


2020.01.15.14:00
     지점  Rep
0    강릉    4
1   강진군   11
2    강화   56
3    거제    3
4    거창   33
..  ...  ...
90   합천   33
91   해남   17
92   홍성   24
93   홍천   63
94  흑산도    0

[95 rows x 2 columns]
2020.01.16.14:00
     지점  Rep
0    강릉    5
1   강진군   12
2    강화   61
3    거제    3
4    거창   37
..  ...  ...
90   합천   37
91   해남   17
92   홍성   28
93   홍천   69
94  흑산도    0

[95 rows x 2 columns]
2020.01.17.14:00
     지점  Rep
0    강릉    5
1   강진군   12
2    강화   65
3    거제    3
4    거창   39
..  ...  ...
90   합천   39
91   해남   18
92   홍성   32
93   홍천   73
94  흑산도    0

[95 rows x 2 columns]
2020.01.18.14:00
     지점  Rep
0    강릉    5
1   강진군   12
2    강화   68
3    거제    3
4    거창   41
..  ...  ...
90   합천   41
91   해남   19
92   홍성   35
93   홍천   76
94  흑산도    0

[95 rows x 2 columns]
2020.01.19.14:00
     지점  Rep
0    강릉    5
1   강진군   13
2    강화   70
3    거제    3
4    거창   44
..  ...  ...
90   합천   44
91   해남   20
92   홍성   35
93   홍천   83
94  흑산도    0

[95 rows x 2 columns]
2020.01.20.14:00
   

2020.02.28.14:00
      0   1
0    강릉  26
1   강진군  35
2    강화  85
3    거제   5
4    거창  52
..  ...  ..
90   합천  45
91   해남  43
92   홍성  65
93   홍천  82
94  흑산도  11

[95 rows x 2 columns]
2020.02.29.14:00
      0   1
0    강릉  26
1   강진군  35
2    강화  85
3    거제   5
4    거창  52
..  ...  ..
90   합천  45
91   해남  43
92   홍성  65
93   홍천  82
94  흑산도  11

[95 rows x 2 columns]
2020.03.01.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  1
94  흑산도  0

[95 rows x 2 columns]
2020.03.02.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  1
94  흑산도  0

[95 rows x 2 columns]
2020.03.03.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  3
94  흑산도  0

[95 rows x 2 columns]
2020.03.04.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0


2020.04.15.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.04.16.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.04.17.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.04.18.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.04.19.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.04.20.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94

2020.06.02.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.06.03.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.06.04.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.06.05.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.06.06.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.06.07.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94

2020.07.20.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.07.21.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.07.22.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.07.23.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.07.24.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.07.25.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94

2020.09.06.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.09.07.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.09.08.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.09.09.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.09.10.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  0

[95 rows x 2 columns]
2020.09.11.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94

2020.10.24.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  1

[95 rows x 2 columns]
2020.10.25.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  2

[95 rows x 2 columns]
2020.10.26.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  2

[95 rows x 2 columns]
2020.10.27.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  2

[95 rows x 2 columns]
2020.10.28.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94  흑산도  2

[95 rows x 2 columns]
2020.10.29.14:00
      0  1
0    강릉  0
1   강진군  0
2    강화  0
3    거제  0
4    거창  0
..  ... ..
90   합천  0
91   해남  0
92   홍성  0
93   홍천  0
94

2020.12.09.14:00
      0   1
0    강릉   0
1   강진군   4
2    강화  15
3    거제   1
4    거창  20
..  ...  ..
90   합천  19
91   해남  19
92   홍성  12
93   홍천  12
94  흑산도  33

[95 rows x 2 columns]
2020.12.10.14:00
      0   1
0    강릉   0
1   강진군   4
2    강화  15
3    거제   1
4    거창  20
..  ...  ..
90   합천  19
91   해남  19
92   홍성  12
93   홍천  12
94  흑산도  33

[95 rows x 2 columns]
2020.12.11.14:00
      0   1
0    강릉   0
1   강진군   4
2    강화  17
3    거제   1
4    거창  21
..  ...  ..
90   합천  20
91   해남  20
92   홍성  12
93   홍천  12
94  흑산도  36

[95 rows x 2 columns]
2020.12.12.14:00
      0   1
0    강릉   0
1   강진군   4
2    강화  20
3    거제   1
4    거창  21
..  ...  ..
90   합천  20
91   해남  20
92   홍성  12
93   홍천  13
94  흑산도  39

[95 rows x 2 columns]
2020.12.13.14:00
      0   1
0    강릉   1
1   강진군   6
2    강화  24
3    거제   1
4    거창  24
..  ...  ..
90   합천  22
91   해남  22
92   홍성  14
93   홍천  13
94  흑산도  48

[95 rows x 2 columns]
2020.12.14.14:00
      0   1
0    강릉  12
1   강진군  13
2    강화  35
3    거제   3
4  

In [ ]:
Result.iloc[:,1] += 7200
Result

In [ ]:
def print_match(m):
# . (ca.e): 하나의 문자를 의미 > care, cafe, case ...
# ^ (^de) : 문자열의 시작 > destination, desk, ,,,
# $ (se$) : 문자열의 끝 > vase, case, base ... 
    if m:
        print("m.group():", m.group()) #일치하는 문자열 반환
        print("m.string:", m.string) #입력받은 문자열
        print("m.start():", m.start()) #일치하는 문자열의 시작 index
        print("m.end():",m.end()) #일치하는 문자열의 시작/끝 index
    else:
        print("매칭되지 않음")
        
p = re.compile("ca.e")
       
#m = p.match("careless") #match: 주어진 문자열의 처음부터 일치하는지 확인
#print_match(m)

#m = p.search("careless") #search: 주어진 문자열 중에 일치하는게 있는지 확인
#print_match(m)

m = p.findall("good care cafe") # findall: 일치하는 모든 것을 리스트 형태로 반환
print(m)

# BeautifulSoup

In [ ]:
soup = BeautifulSoup(res.text,"lxml")
print(soup.title) #soup 객체에서 처음 발견되는 title 객체 출력
print(soup.title.get_text()) # title 객체에서 내용만 출력

In [ ]:
print(soup.a) # soup 객체에서 처음 발견되는 a element 출력
print(soup.a.attrs) # a element의 속성 정보 출력
print(soup.a["class"]) # a element의 href 속성 '값' 정보를 출력

In [ ]:
#soup.find("table", attrs={"class" : "table_develop3"})
test = soup.find("tbody")
#test = test.tr.td.a.get_text()
print(test.tr.td.a.next_sibling)

In [ ]:
soup.select("tbody>tr>td>a")

In [ ]:
table = soup.select('table')
table = str(table[0])
table_df_list = pd.read_html(table)
table_df = table_df_list[0]
table_df